<h1 style="text-align: center;"> <b> Advanced Fixed Income Case Study </b>
<P> "Deutsche Bank: Finding Relative Value Trades"  
</h1>



#### Maël Thuillier - 79372, Solal Fuster - 72950, Grigory Sheleg - 80985, Felix Tremblot de La Croix - 87510, Elyazid Benkhadra - 80574,

#### First of all, let's install and import the required libraries:

In [1]:
#Check if the require libraries are installed and install them if not
#import sys

#!py -m pip install -r requirements.txt

In [2]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import datetime

import QuantLib as ql
from scipy.optimize import linprog
from scipy.optimize import least_squares
from scipy.interpolate import CubicSpline

from nelson_siegel_svensson import NelsonSiegelSvenssonCurve
from nelson_siegel_svensson import NelsonSiegelCurve
from nelson_siegel_svensson.calibrate import calibrate_nss_ols

import graph as gr

In [3]:
# For better readibility we will always show only the first and last 6 rows of huge dataframe, run pd.reset_option to revert to default
pd.set_option('display.max_rows', 13)

#pd.reset_option('display.max_rows')

#### Data importation and cleaning for Exhibit 1 (df):

In [4]:
# We set a random state globally for our calculations
np.random.seed(2**32 - 1)

# Extract data
df = pd.read_excel(
    r"XLS698-XLS-ENG.xls",
    sheet_name="Exhibit 1",
    header=2
)

StartDate = datetime.datetime(2003, 8, 15)
df['Starting Date'] = pd.to_datetime('2003-08-15')

df['Maturity Date']=pd.to_datetime(df['Maturity Date'])

df['Time to Maturity'] = (df['Maturity Date'] - StartDate)/ pd.Timedelta(days=1) #change format to get Time to Maturity in years float
df['Time to Maturity'] = (df['Time to Maturity'] / 365.25).round(1)

df.dropna(subset=["Maturity Date", "Current Price"], inplace=True)
df.reset_index(drop=True, inplace=True)

df.head()

,Coupon Rate (%),Maturity Date,Current Price,Starting Date,Time to Maturity
0,3,2004-02-15,101.0544,2003-08-15,0.5
1,2.125,2004-08-15,100.9254,2003-08-15,1.0
2,1.5,2005-02-15,99.8942,2003-08-15,1.5
3,6.5,2005-08-15,109.0934,2003-08-15,2.0
4,5.625,2006-02-15,108.4380,2003-08-15,2.5


#### Question 1.1

In [5]:
#Quantlib is a free/open-source library for quantitative finance based on C++ and used in Python that we will use for the rest of the assignement
#as it is a very powerful library and it is easy to use 

# We assume face value is $100 for all bonds
face_value = 100

# We prepare the bond data for the QuantLib package
df['Coupon Rate'] = df['Coupon Rate (%)'] / 100
df['Quote'] = df['Current Price'] 

#  We set evaluation date with the one we have in the case study
ql.Settings.instance().evaluationDate = ql.Date(15, 8, 2003)

# We define the calendar and day count convention (important for QuantLib, 
# here we have U.S. government bonds, so the U.S. government bond calendar prevails 
# and the day count is based on the ISDA convention)
calendar = ql.UnitedStates(ql.UnitedStates.GovernmentBond)
day_count = ql.ActualActual(ql.ActualActual.ISDA)
business_convention = ql.Unadjusted

# Define the compounding and frequency
compounding = ql.Compounded
compounding_frequency = ql.Semiannual

# Create an empty list to store the bond helpers
bond_helpers = []

# Flag for identifying the first two bonds
is_first_bond = True

for index, row in df.iterrows():
    # Define bond terms
    issue_date = ql.Date(15, 8, 2003)
    maturity_date = ql.Date(row['Maturity Date'].day, row['Maturity Date'].month, row['Maturity Date'].year)
    tenor = ql.Period(ql.Semiannual)
    settlement_days = 0
    face_amount = face_value
    coupon_rate = [row['Coupon Rate']]
    coupons = coupon_rate
    accrual_day_count = ql.ActualActual(ql.ActualActual.ISDA)  # Specify the convention
    payment_convention = ql.Unadjusted

    # Create a schedule for the bond
    schedule = ql.Schedule(
        issue_date,
        maturity_date,
        tenor,
        calendar,
        ql.Unadjusted,
        ql.Unadjusted,
        ql.DateGeneration.Backward,
        False
    )

    # Calculate accrued interest
    bond = ql.FixedRateBond(
        settlement_days,
        face_amount,
        schedule,
        coupons,
        accrual_day_count,
        payment_convention
    )
    
    accrued_interest = bond.accruedAmount(ql.Settings.instance().evaluationDate)

    # Adjust the price for the first two bonds with remaining coupon payments
    if is_first_bond or index == 1:
        # Account for the fact that these are old coupon bonds; accrued interest
        clean_price = row['Quote']
        full_price = clean_price + accrued_interest
        quote_handle = ql.QuoteHandle(ql.SimpleQuote(full_price))
    else:
        # For the rest of the bonds, use the clean price (no coupon adjustments needed)
        quote_handle = ql.QuoteHandle(ql.SimpleQuote(row['Quote']))

    bond_helper = ql.FixedRateBondHelper(
        quote_handle,
        settlement_days,
        face_amount,
        schedule,
        coupons,
        accrual_day_count,
        payment_convention
    )

    bond_helpers.append(bond_helper)

    # Set flag to false after processing the first two bonds
    is_first_bond = False

# Build the yield curve
yieldcurve = ql.PiecewiseLogCubicDiscount(
    ql.Settings.instance().evaluationDate,
    bond_helpers,
    day_count
)

zero_rates = []
maturities = []

# Loop through each bond to extract zero-coupon rates
for index, row in df.iterrows():
    maturity_date = ql.Date(row['Maturity Date'].day, row['Maturity Date'].month, row['Maturity Date'].year)
    ttm = day_count.yearFraction(ql.Settings.instance().evaluationDate, maturity_date)
    rate = yieldcurve.zeroRate(
        ttm,
        compounding,
        compounding_frequency
        
    ).rate()

    zero_rates.append(rate * 100)
    maturities.append(ttm)


df['Zero-Coupon Yield'] = zero_rates
df['Zero-Coupon Yield'] = df['Zero-Coupon Yield']

df

,Coupon Rate (%),Maturity Date,Current Price,Starting Date,Time to Maturity,Coupon Rate,Quote,Zero-Coupon Yield
0,3,2004-02-15,101.0544,2003-08-15,0.5,0.03,101.0544,0.897514
1,2.125,2004-08-15,100.9254,2003-08-15,1.0,0.02125,100.9254,1.193858
2,1.5,2005-02-15,99.8942,2003-08-15,1.5,0.015,99.8942,1.575041
3,6.5,2005-08-15,109.0934,2003-08-15,2.0,0.065,109.0934,1.872713
4,5.625,2006-02-15,108.4380,2003-08-15,2.5,0.05625,108.4380,2.179647
...,...,...,...,...,...,...,...,...
47,6.375,2027-08-15,111.4036,2003-08-15,24.0,0.06375,111.4036,5.960242
48,6.125,2028-02-15,108.0391,2003-08-15,24.5,0.06125,108.0391,5.955634
49,5.5,2028-08-15,99.6330,2003-08-15,25.0,0.055,99.6330,5.912716
50,5.25,2029-02-15,96.2876,2003-08-15,25.5,0.0525,96.2876,5.878123


#### Question 1.2
####
#### Data importation and cleaning for the Exhibit 4 (df_db):

In [6]:
# Data importation and cleaning for sheet exhibit_4
df_db = pd.read_excel(
    r"XLS698-XLS-ENG.xls",
    sheet_name="Exhibit 4 ",
    header=2 
)

df_db['Time to Maturity'] = pd.to_numeric(df_db['Maturity (years)'].str.replace('y', ''), errors='coerce')
df_db = df_db.drop(columns='Maturity (years)')

df_db['Model Prediction (BEY)'] = pd.to_numeric(df_db['Model Prediction (BEY)'].str.replace('%', ''), errors='coerce')

df_db.dropna(subset=["Time to Maturity", "Model Prediction (BEY)"], inplace=True)
df_db.reset_index(drop=True, inplace=True)

df_db

,Model Prediction (BEY),Time to Maturity
0,1.2443,1.0
1,1.8727,2.0
2,2.4110,3.0
3,2.9665,4.0
4,3.4454,5.0
5,3.8557,6.0
6,4.1996,7.0
7,4.4677,8.0
8,4.6528,9.0
9,4.7107,10.0


In [7]:
Yield = df[['Zero-Coupon Yield','Time to Maturity']]
DBvsYield = pd.merge(df_db, Yield, on='Time to Maturity', how='inner').round(4)

gr.Graph_Curve(Yield,DBvsYield.round(4))

In [8]:
DBvsYield['Buy or Sell ?']= np.where(DBvsYield['Model Prediction (BEY)'] < DBvsYield['Zero-Coupon Yield'], "Buy",
                            np.where(DBvsYield['Model Prediction (BEY)'] > DBvsYield['Zero-Coupon Yield'], "Sell", 
                            "Hold"))

new_order = ['Time to Maturity', 'Model Prediction (BEY)', 'Zero-Coupon Yield', 'Buy or Sell ?']

DBvsYield = DBvsYield.reindex(columns=new_order)

DBvsYield

,Time to Maturity,Model Prediction (BEY),Zero-Coupon Yield,Buy or Sell ?
0,1.0,1.2443,1.1939,Sell
1,2.0,1.8727,1.8727,Hold
2,3.0,2.4110,2.4711,Buy
3,4.0,2.9665,3.0083,Buy
4,5.0,3.4454,3.4798,Buy
5,6.0,3.8557,3.8817,Buy
6,7.0,4.1996,4.2203,Buy
7,8.0,4.4677,4.4865,Buy
8,9.0,4.6528,4.6640,Buy
9,10.0,4.7107,4.7107,Hold


#### Question 2.1:

In [9]:
#Define parameters for the Cubic Spline interpolation
DataCubic = DBvsYield[DBvsYield['Time to Maturity'].isin([2,5,10,15,20])]  #5 knots
X_Cubic = DataCubic['Time to Maturity']
Y_Cubic = DataCubic['Zero-Coupon Yield']/100

maturity_range = np.linspace(0.5, 26, 100)

cs = CubicSpline(X_Cubic, Y_Cubic, bc_type='natural') #interpolation

x_newQ1 = maturity_range
y_newQ1 = cs(x_newQ1)*100

In [10]:
gr.Graph_EstimationVSZero(Yield,Yield['Zero-Coupon Yield'],x_newQ1,y_newQ1,'Zero-Coupon Yield Curve over 25 Years','Cubic Splines')

In [11]:
# Calculate the discount function using the interpolated zero-coupon yields
discount_function = 1 / ((1 + cs(maturity_range)) ** maturity_range)

gr.graph_discount(maturity_range, discount_function,"Discount Function (Cubic Splines)","Discount Function Cubic")

#### Question 2.2

In [12]:
#Decision with new interpolated zerp-coupon yield curve
specified_maturities = DBvsYield['Time to Maturity']
spline_yields = cs(specified_maturities) * 100
spline_yields = spline_yields.round(4)

df_spline_yields = pd.DataFrame({
    'Time to Maturity': specified_maturities,
    'Spline Zero-Coupon Yield': spline_yields
})

df_merged = pd.merge(DBvsYield, df_spline_yields, on='Time to Maturity', how='inner')

df_merged['Buy or Sell ?']= np.where(df_merged['Model Prediction (BEY)'] < df_merged['Spline Zero-Coupon Yield'], "Buy",
                            np.where(df_merged['Model Prediction (BEY)'] > df_merged['Spline Zero-Coupon Yield'], "Sell", 
                            "Hold"))

df_merged[['Time to Maturity', 'Spline Zero-Coupon Yield', 'Model Prediction (BEY)', 'Buy or Sell ?']]

,Time to Maturity,Spline Zero-Coupon Yield,Model Prediction (BEY),Buy or Sell ?
0,1.0,1.2847,1.2443,Buy
1,2.0,1.8727,1.8727,Hold
2,3.0,2.4607,2.4110,Buy
3,4.0,3.0095,2.9665,Buy
4,5.0,3.4798,3.4454,Buy
5,6.0,3.8438,3.8557,Sell
6,7.0,4.1195,4.1996,Sell
7,8.0,4.3362,4.4677,Sell
8,9.0,4.5235,4.6528,Sell
9,10.0,4.7107,4.7107,Hold


In [13]:
gr.Graph_EstimationVSZero(df_merged,df_merged['Spline Zero-Coupon Yield'],df_merged['Time to Maturity'],df_merged['Model Prediction (BEY)'],'Comparison of Spline-Based Zero-Coupon Yields with Model Predictions','Model BEY')

#### Question 2.3

In [14]:
#Define parameters for the Cubic Spline interpolation with 2 Knots now
DataCubic2 = DBvsYield[DBvsYield['Time to Maturity'].isin([5,20])]
X_Cubic = DataCubic2['Time to Maturity']
Y_Cubic = DataCubic2['Zero-Coupon Yield']/100

cs2 = CubicSpline(X_Cubic, Y_Cubic, bc_type='natural', extrapolate=True)

x_newQ2 = maturity_range
y_newQ2 = cs2(x_newQ2)*100

In [15]:
gr.Graph_EstimationVSZero(Yield,Yield['Zero-Coupon Yield'],x_newQ2,y_newQ2,'Zero-Coupon Yield Curve, 2 nods Cubic Spline','Cubic Spline')

In [16]:
discount_function2 = 1 / ((1 + cs2(maturity_range)) ** maturity_range)

gr.graph_discount2(maturity_range, discount_function, maturity_range, discount_function2,"Q2.3 Discount Function (Cubic Splines)", "Discount Factors",'Discount Function Cubic Splines', 'Q2.3 Discount Function Cubic Splines')

#### Question 3.1:

In [17]:
# Extract maturities and zero-coupon yields from df
t = df['Time to Maturity'].values
y = df['Zero-Coupon Yield'].values / 100  

# Model calibration
curve, status = calibrate_nss_ols(t, y)
assert status.success #check if not any error

print("Nelson-Siegel-Svensson Curve Parameters:")
print(f"{'Beta 0':<10}: {curve.beta0:.5f}")
print(f"{'Beta 1':<10}: {curve.beta1:.5f}")
print(f"{'Beta 2':<10}: {curve.beta2:.5f}")
print(f"{'Beta 3':<10}: {curve.beta3:.5f}")
print(f"{'Tau 1':<10}: {curve.tau1:.5f}")
print(f"{'Tau 2':<10}: {curve.tau2:.5f}")

Nelson-Siegel-Svensson Curve Parameters:
Beta 0    : 0.06033
Beta 1    : -0.05391
Beta 2    : -0.06016
Beta 3    : 0.07206
Tau 1     : 3.14798
Tau 2     : 5.15351


In [18]:
y_model = curve
y_model_values = y_model(t)

discount_factors = np.exp(-y_model_values * t)

In [19]:
# Plot the observed zero-coupon yields and the NSS model yields
gr.Graph_EstimationVSZero(Yield,Yield['Zero-Coupon Yield'],t,y_model_values*100,'Nelson-Siegel-Svensson Model Fit to Zero-Coupon Yields','NSS Model Yields')

In [20]:
# Plot the discount function estimated via the NSS model
gr.graph_discount(maturity_range, discount_function,"Discount Function (NSS)","Discount Function NSS")

#### Question 3.2:

In [21]:
# Plot the zero-coupon yield curve estimated via the NSS model and Cubic Spline
gr.graph_discount2(maturity_range, cs(maturity_range) * 100,t , y_model_values * 100,'Nelson-Siegel-Svensson Model Fit to Zero-Coupon Yields','Yield (%)','Q2.1 Cubic Splines Yield Curve','Q3.1 NSS Yield Curve')

#### Question 3.3:

In [22]:
# Calculate factor loadings for each maturity
def factor_loadings(t, tau1, tau2):
    
    loading1 = np.ones_like(t)
    loading2 = ((1 - np.exp(-t / tau1)) / (t / tau1))
    loading3 = (((1 - np.exp(-t / tau1)) / (t / tau1)) - np.exp(-t / tau1))
    loading4 = (((1 - np.exp(-t / tau2)) / (t / tau2)) - np.exp(-t / tau2))
    
    return loading1, loading2, loading3, loading4

In [23]:
# Extract estimated parameters from the curve obtained in Q3.1
beta0_est = curve.beta0
beta1_est = curve.beta1
beta2_est = curve.beta2
beta3_est = curve.beta3
tau1_est = curve.tau1
tau2_est = curve.tau2

# Define maturities for plotting (fine grid for smooth curves)
t_plot = np.linspace(0.1, max(t), 200)  

In [24]:
loading1, loading2, loading3, loading4 = factor_loadings(t_plot, tau1_est, tau2_est)

# Calculate contributions from each factor
contribution1 = beta0_est * loading1
contribution2 = beta1_est * loading2
contribution3 = beta2_est * loading3
contribution4 = beta3_est * loading4

# Calculate the total model yield for plotting
y_model_plot = contribution1 + contribution2 + contribution3 + contribution4

In [25]:
# Compute factor loadings in the NSS model
gr.plot_factor_loadings(t_plot, loading1, loading2, loading3, loading4)

In [26]:
# Compute Contributions of each factor in the NSS model
gr.plot_yield_contributions(t_plot, contribution1, contribution2, contribution3, contribution4, y_model_plot)

In [27]:
# Compute Contributions of each factor in the NSS model
gr.Graph_EstimationVSZero(Yield,Yield['Zero-Coupon Yield'],t,y_model_values*100,'Observed Zero-Coupon Yields vs NSS Model Yields','NSS Model Yields')

#### Question 4.1

In [28]:
#Prepartion of data for the question
knots_maturities = [2, 5, 10, 15, 20]
DataKRD = Yield[Yield['Time to Maturity'].isin(knots_maturities)]

original_zero_coupon_yields = DataKRD['Zero-Coupon Yield'].copy()/ 100
zero_coupon_yields_shocked = DataKRD['Zero-Coupon Yield'].copy()/ 100

zero_coupon_yields_shocked.reset_index(drop=True, inplace=True)
original_zero_coupon_yields.reset_index(drop=True, inplace=True)

zero_coupon_yields_shocked.iloc[2] += 0.005
zero_coupon_yields_shocked = zero_coupon_yields_shocked.round(6)
zero_coupon_yields_shocked

0    0.018727
1    0.034798
2    0.052107
3    0.057166
4    0.059512
Name: Zero-Coupon Yield, dtype: float64

In [29]:
#Interpolate to see impact of the shock
cs_original = CubicSpline(DataKRD['Time to Maturity'], original_zero_coupon_yields, bc_type='natural')
cs_shocked = CubicSpline(DataKRD['Time to Maturity'], zero_coupon_yields_shocked, bc_type='natural')

maturity_range = np.linspace(0.5, 20, 100)

original_yields_interp = cs_original(maturity_range)
shocked_yields_interp = cs_shocked(maturity_range)

yield_difference = (shocked_yields_interp - original_yields_interp) * 10000  # Convert to basis points

In [30]:
# Plot the original and shocked zero-coupon yield curves
gr.plot_shocked_vs_original_yields(DataKRD, original_zero_coupon_yields, zero_coupon_yields_shocked, maturity_range, original_yields_interp, shocked_yields_interp)

In [31]:
#Plot the difference between the original and shocked zero-coupon yield curves
gr.plot_yield_difference(maturity_range, yield_difference)

#### Question 4.2
##### First of all, we create a function that allow us to compute the bond's present values:

In [32]:
def Bond_Present_Value(Coupon_Rate_Simple, maturity, zero_coupon_rates):
    
    """
    Calculate the present value of a bond with semiannual coupons using zero-coupon bond rates.
    
    Parameters:
    - face_value (float): The face value of the bond.
    - coupon_rate (float): The annual coupon rate as a decimal (e.g., 0.05 for 5%).
    - maturity (float): The maturity of the bond in years.
    - zero_coupon_rates (pd.DataFrame): A DataFrame with two columns - 'maturity' and 'rate'.
    
    Returns:
    - float: The present value of the bond.
    """

    num_periods = int(maturity * 2)
    pv_bond = 0
    Coupon_Rate_Simple = Coupon_Rate_Simple/200

    for period in range(1, num_periods): 
        period_years = period / 2
        rate = zero_coupon_rates.loc[zero_coupon_rates['Time to Maturity'] == period_years, 'Zero-Coupon Yield'].values
        discount_factor = 1 / ((1 + rate[0]/200) ** period)
        pv_bond += Coupon_Rate_Simple * 100 * discount_factor

    rate = zero_coupon_rates.loc[zero_coupon_rates['Time to Maturity'] == maturity, 'Zero-Coupon Yield'].values
    discount_factor = 1 / ((1 + rate[0]/200) ** num_periods)
    
    pv_bond += 100 * discount_factor + 100 * Coupon_Rate_Simple * discount_factor

    return pv_bond

In [33]:
weight = 1000000/len(Yield)

In [34]:
#Define a variable representing the number of bonds bought with 10000000/52 = 192307.69
df['Numbers of Bonds'] = weight/df['Current Price']

In [35]:
# Compute value of portfolio without shock and not taking account of weight for now
sum = 0 
count = 0

for bond in df['Time to Maturity']:
    sum += Bond_Present_Value(df['Coupon Rate (%)'].iloc[count], bond, Yield)*df['Numbers of Bonds'].iloc[count]
    count += 1

In [36]:
#We run over each bond in the portfolio to calculate its present value with shocked yield (10 years)
sum_shock = 0 
count = 0

Yield_Shocked = Yield.copy()
Yield_Shocked.loc[19,['Zero-Coupon Yield']] = Yield_Shocked['Zero-Coupon Yield'].loc[19] + 0.5

for bond in df['Time to Maturity']:
        
    if 10 <= bond:
        sum_shock += Bond_Present_Value(df['Coupon Rate (%)'].iloc[count], bond, Yield_Shocked)*df['Numbers of Bonds'].iloc[count]
    else:
        sum_shock += Bond_Present_Value(df['Coupon Rate (%)'].iloc[count], bond, Yield)*df['Numbers of Bonds'].iloc[count]
            
    count += 1


In [37]:
print('Value of KRD for a shock of 50bps of the 10-year rate')
print('$' + str(round((sum_shock-sum),2)))

Value of KRD for a shock of 50bps of the 10-year rate
$-1169.34


#### Question 4.3
##### -50bps curve shock for all maturities:

In [38]:
Sum_Shocked = 0
Shocked = []
Not_Shocked = []
Sum = 0 

count = 0
shift = -0.5

# Compute value of portfolio for each shocked maturity
for Maturity in Yield['Time to Maturity']:
    Yield_Shocked = Yield.copy()
    Yield_Shocked.loc[count,['Zero-Coupon Yield']] = Yield_Shocked['Zero-Coupon Yield'].loc[count] + shift
    count_bond = 0
    Sum_Shocked = 0
    Sum_Not_Shocked = 0
    
    for bond in df['Time to Maturity']:
        if Maturity <= bond:
            Sum_Shocked += Bond_Present_Value(df['Coupon Rate (%)'].iloc[count_bond], bond, Yield_Shocked)*df['Numbers of Bonds'].iloc[count_bond]
        else:
            Sum_Shocked += Bond_Present_Value(df['Coupon Rate (%)'].iloc[count_bond], bond, Yield)*df['Numbers of Bonds'].iloc[count_bond]

        Sum_Not_Shocked += Bond_Present_Value(df['Coupon Rate (%)'].iloc[count_bond], bond, Yield)*df['Numbers of Bonds'].iloc[count_bond]

        count_bond += 1

    count += 1

    Not_Shocked.append(Sum_Not_Shocked) 
    Shocked.append(Sum_Shocked) 

In [39]:
KRD = [(Shocked[i]-Not_Shocked[i]) for i in range(len(Shocked))]
KRD[19]

np.float64(1230.880474363803)

In [40]:
gr.plot_key_rate_durations(df, KRD, weight)

##### +50bps curve shock:

In [41]:
Sum_Shocked = 0
Shocked = []
Not_Shocked = []
Sum = 0 

count = 0
shift = +0.5

# Compute value of portfolio for each shocked maturity
for Maturity in Yield['Time to Maturity']:
    Yield_Shocked = Yield.copy()
    Yield_Shocked.loc[count,['Zero-Coupon Yield']] = Yield_Shocked['Zero-Coupon Yield'].loc[count] + shift
    count_bond = 0
    Sum_Shocked = 0
    Sum_Not_Shocked = 0
    
    for bond in df['Time to Maturity']:
        if Maturity <= bond:
            Sum_Shocked += Bond_Present_Value(df['Coupon Rate (%)'].iloc[count_bond], bond, Yield_Shocked)*df['Numbers of Bonds'].iloc[count_bond]
        else:
            Sum_Shocked += Bond_Present_Value(df['Coupon Rate (%)'].iloc[count_bond], bond, Yield)*df['Numbers of Bonds'].iloc[count_bond]

        Sum_Not_Shocked += Bond_Present_Value(df['Coupon Rate (%)'].iloc[count_bond], bond, Yield)*df['Numbers of Bonds'].iloc[count_bond]

        count_bond += 1

    count += 1

    Not_Shocked.append(Sum_Not_Shocked) 
    Shocked.append(Sum_Shocked) 

In [42]:
KRD = [(Shocked[i]-Not_Shocked[i]) for i in range(len(Shocked))]

In [43]:
gr.plot_key_rate_durations(df, KRD, weight)

#### Question 5.1

In [44]:
average_maturity = np.mean(df["Time to Maturity"].values)
random_noise = np.random.normal(0, 0.0025, size=len(df))

df["Steepened Yield"] = df["Zero-Coupon Yield"] / 100 + ((df["Time to Maturity"] - average_maturity) / average_maturity) * 0.005 + random_noise
df["Deterministic Component"] = df["Zero-Coupon Yield"] / 100 + ((df["Time to Maturity"] - average_maturity) / average_maturity) * 0.005
df["Stochastic Component"] = random_noise

In [45]:
gr.Graph_EstimationVSZero(Yield,Yield["Zero-Coupon Yield"],Yield["Time to Maturity"],df["Steepened Yield"]*100,'Steepened Yield Curve vs Original Zero-Coupon Yield Curve','Steepened Yield Curve')

In [46]:
gr.graph_discount2(df["Time to Maturity"],df["Deterministic Component"] * 100,df["Time to Maturity"],df["Stochastic Component"] * 100,'Deterministic vs Stochastic Components of the Steepened Yield Curve','Contribution (bps)','Deterministic Component','Stochastic Component (ε)')

#### Question 5.2

In [47]:
t = df['Time to Maturity'].values
y = df["Steepened Yield"].values

# Model calibration
curve, status = calibrate_nss_ols(t, y)
assert status.success

In [48]:
print("Nelson-Siegel-Svensson Curve Parameters:")

print(f"{'Beta 0':<10}: {curve.beta0:.5f}")
print(f"{'Beta 1':<10}: {curve.beta1:.5f}")
print(f"{'Beta 2':<10}: {curve.beta2:.5f}")
print(f"{'Beta 3':<10}: {curve.beta3:.5f}")
print(f"{'Tau 1':<10}: {curve.tau1:.5f}")
print(f"{'Tau 2':<10}: {curve.tau2:.5f}")

Nelson-Siegel-Svensson Curve Parameters:
Beta 0    : 0.06886
Beta 1    : -0.06603
Beta 2    : -0.06592
Beta 3    : 0.05366
Tau 1     : 2.99678
Tau 2     : 5.02365


In [49]:
y_model = curve
y_model_values = y_model(t)

discount_factors = np.exp(-y_model_values * t)

In [50]:
gr.Graph_EstimationVSZero(Yield,Yield['Zero-Coupon Yield'],t,y_model_values*100,'Nelson-Siegel-Svensson Model Fitted with Steepened Yield Compared to Zero-Coupon Yields','NSS Model Yields')

In [51]:
gr.graph_discount(t, discount_factors,'Discount Function Estimated via Nelson-Siegel-Svensson Model',"Discount Function NSS")

In [52]:
# Function to price a bond using NSS discount factors
def price_bond_nss(bond):
    
    cashflows = bond.cashflows()
    pv = 0.0
    
    for cf in cashflows:
        if cf.date() > ql.Settings.instance().evaluationDate:
            t_cf = round(day_count.yearFraction(ql.Settings.instance().evaluationDate, cf.date()),2)
            df = discount_factor_dict.get(t_cf, None)
            if df is not None:
                pv += cf.amount() * df
            else:
                raise ValueError(f"No discount factor found for cash flow time {t_cf}")
    return pv

In [53]:
ql.Settings.instance().evaluationDate = ql.Date(15, 8, 2003)
calendar = ql.UnitedStates(ql.UnitedStates.GovernmentBond)
day_count = ql.ActualActual(ql.ActualActual.ISDA)

# Create a dictionary mapping each time `t` to its corresponding discount factor
discount_factor_dict = dict(zip(t, discount_factors))

bonds_nss_prices = []

for index, row  in df.iterrows():

    issue_date = ql.Date(15, 8, 2003)
    maturity_date = ql.Date(row['Maturity Date'].day, row['Maturity Date'].month, row['Maturity Date'].year)
    tenor = ql.Period(ql.Semiannual)
    settlement_days = 0
    face_amount = 100
    coupon_rate = [row['Coupon Rate']]
    accrual_day_count = ql.ActualActual(ql.ActualActual.ISDA)
    payment_convention = ql.Unadjusted

    schedule = ql.Schedule(
        issue_date,
        maturity_date,
        tenor,
        calendar,
        ql.Unadjusted,
        ql.Unadjusted,
        ql.DateGeneration.Backward,
        False
    )

    bond = ql.FixedRateBond(
        settlement_days,
        face_amount,
        schedule,
        coupon_rate,
        accrual_day_count,
        payment_convention
    )

    nss_price = price_bond_nss(bond)
    
    bonds_nss_prices.append(nss_price)

df['NSS Theoretical Price'] = bonds_nss_prices
df['NSS Theoretical Price'] = df['NSS Theoretical Price'].round(4)

df['Price Difference'] = df['NSS Theoretical Price'] - df['Current Price']

# Identify cheap and expensive bonds
df['Valuation'] = np.where(
    df['Price Difference'] > 0,
    'Cheap',
    'Expensive'
)

In [54]:
# Display results
display_columns = ['Maturity Date', 'Coupon Rate (%)', 'Current Price', 'NSS Theoretical Price', 'Price Difference', 'Valuation']
df[display_columns].head()

,Maturity Date,Coupon Rate (%),Current Price,NSS Theoretical Price,Price Difference,Valuation
0,2004-02-15,3,101.0544,101.2263,0.1719,Cheap
1,2004-08-15,2.125,100.9254,101.2617,0.3363,Cheap
2,2005-02-15,1.5,99.8942,100.5103,0.6161,Cheap
3,2005-08-15,6.5,109.0934,109.9289,0.8355,Cheap
4,2006-02-15,5.625,108.4380,109.4864,1.0484,Cheap


#### Question 5.3


In [55]:
# For cheap bonds
cheap_bonds = df[df['Valuation'] == 'Cheap'].reset_index(drop=True)
cheap_bonds['EP_per_Dollar'] = cheap_bonds['Price Difference'] / cheap_bonds['Current Price']

# For expensive bonds
expensive_bonds = df[df['Valuation'] == 'Expensive'].reset_index(drop=True)
expensive_bonds['EP_per_Dollar'] = (-expensive_bonds['Price Difference']) / expensive_bonds['Current Price']

# Calculate total expected profit per dollar for each group
total_expected_profit_cheap = cheap_bonds['EP_per_Dollar'].sum()
total_expected_profit_expensive = expensive_bonds['EP_per_Dollar'].sum()

# Allocate Investments within each group to sum up to $100
cheap_bonds['Investment'] = (cheap_bonds['EP_per_Dollar'] / total_expected_profit_cheap) * 100
expensive_bonds['Investment'] = (expensive_bonds['EP_per_Dollar'] / total_expected_profit_expensive) * 100

In [56]:
print("Investment Allocation for Cheap Bonds:")

display(
    cheap_bonds[display_columns].style.format({
        'Coupon Rate (%)': '{:.3f}',
        'Current Price': '{:.2f}',
        'Time to Maturity': '{:.1f}',
        'NSS Theoretical Price': '{:.2f}',
        'Price Difference': '{:.2f}',
        'EP_per_Dollar': '{:.5f}',
        'Investment': '{:.4f}%',
        'Starting Date': '{:%Y-%m-%d}',
        'Maturity Date': '{:%Y-%m-%d}'
    }).set_caption("Cheap Bonds")  
)

Investment Allocation for Cheap Bonds:


,Maturity Date,Coupon Rate (%),Current Price,NSS Theoretical Price,Price Difference,Valuation
0,2004-02-15,3.000,101.05,101.23,0.17,Cheap
1,2004-08-15,2.125,100.93,101.26,0.34,Cheap
2,2005-02-15,1.500,99.89,100.51,0.62,Cheap
3,2005-08-15,6.500,109.09,109.93,0.84,Cheap
4,2006-02-15,5.625,108.44,109.49,1.05,Cheap
5,2006-08-15,2.375,99.78,100.94,1.16,Cheap
6,2007-02-15,6.250,111.72,113.28,1.56,Cheap
7,2007-08-15,3.250,101.08,102.51,1.43,Cheap
8,2008-02-15,3.000,99.17,100.67,1.50,Cheap
9,2008-08-15,3.250,99.27,100.80,1.53,Cheap


In [57]:
print("\nInvestment Allocation for Expensive Bonds:")

display(
    expensive_bonds[display_columns].style.format({
        'Coupon Rate (%)': '{:.3f}',
        'Current Price': '{:.2f}',
        'Time to Maturity': '{:.1f}',
        'NSS Theoretical Price': '{:.2f}',
        'Price Difference': '{:.2f}',
        'EP_per_Dollar': '{:.5f}',
        'Investment': '{:.4f}%',
        'Starting Date': '{:%Y-%m-%d}',
        'Maturity Date': '{:%Y-%m-%d}'
    }).set_caption("Expensive Bonds")
)


Investment Allocation for Expensive Bonds:


,Maturity Date,Coupon Rate (%),Current Price,NSS Theoretical Price,Price Difference,Valuation
0,2013-02-15,3.875,95.03,94.52,-0.51,Expensive
1,2013-08-15,4.250,97.77,96.53,-1.24,Expensive
2,2014-02-15,13.250,174.33,172.96,-1.36,Expensive
3,2014-08-15,12.500,168.94,167.90,-1.04,Expensive
4,2018-02-15,9.125,140.79,140.57,-0.22,Expensive
5,2018-08-15,9.000,139.91,139.53,-0.38,Expensive
6,2019-02-15,8.875,138.74,138.46,-0.28,Expensive
7,2019-08-15,8.125,130.72,130.30,-0.41,Expensive
8,2020-02-15,8.500,135.29,134.70,-0.59,Expensive
9,2020-08-15,8.750,138.35,137.77,-0.57,Expensive


In [58]:

total_investment = cheap_bonds['Investment'].sum() + expensive_bonds['Investment'].sum()
print(f"\nTotal Investment Allocated: ${total_investment:.2f}")

# Calculate Total Expected Profit and Return
total_expected_profit_long = (cheap_bonds['EP_per_Dollar'] * cheap_bonds['Investment']).sum()
print(f"\nTotal Long Expected Profit: ${total_expected_profit_long:.2f}")
total_expected_profit_short = (expensive_bonds['EP_per_Dollar'] * expensive_bonds['Investment']).sum()
print(f"\nTotal Short Expected Profit: ${total_expected_profit_short:.2f}")
net_expected_return = total_expected_profit_long + total_expected_profit_short
expected_return_percentage = (net_expected_return / 200) * 100 

print(f"\nTotal Expected Profit: ${net_expected_return:.2f}")
print(f"Expected Return Percentage: {expected_return_percentage:.2f}%")


Total Investment Allocated: $200.00

Total Long Expected Profit: $1.16

Total Short Expected Profit: $2.30

Total Expected Profit: $3.46
Expected Return Percentage: 1.73%
